<a href="https://colab.research.google.com/github/sai5979/sai-revanth/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install  scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=4ce6ad106642224905d2649daec45c5e2228b841cb82bc4e3acd44221f9567de
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [9]:
pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 15.0 MB/s 


In [12]:
pip install pickle-mixin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6006 sha256=c4392503ee331df93d8097e4e099232dfbefe15caf03d0609b48c75d117aaf13
  Stored in directory: /root/.cache/pip/wheels/2a/a4/6c/83bfbc3b94f1bb43d634b07a6a893fd437a45c58b29aea5142
Successfully built pickle-mixin


In [27]:
## Classifer-1
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing import image
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing import image
from tensorflow import keras
from keras.layers import Input,Lambda,Dense,Flatten
import os
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Model, load_model
from keras.layers import Dense, Input, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
Image_Size=[224,224]
train_path='chest_xray/train'
test_path='chest_xray/test'
def load_normal(norm_path):
  norm_files = np.array(os.listdir(norm_path))
  norm_labels = np.array(['normal']*len(norm_files))
  norm_images = []
  for image in tqdm(norm_files):
    image = cv2.imread(norm_path + image)
    image = cv2.resize(image, dsize=(224,224))
    norm_images.append(image)
    norm_images = np.array(norm_images)
  return norm_images, norm_labels
def load_pneumonia(pneu_path):
  pneu_files = np.array(os.listdir(pneu_path))
  pneu_labels = np.array(['pneumonia']*len(pneu_files))
  pneu_images = []
  for image in tqdm(pneu_files):
    image = cv2.imread(pneu_path + image)
    image = cv2.resize(image, dsize=(224,224))
    pneu_images.append(image)  
    pneu_images = np.array(pneu_images)
  return pneu_images, pneu_labels
  norm_images, norm_labels = load_normal('chest_xray/train/NORMAL/')
  pneu_images, pneu_labels = load_pneumonia('chest_xray/train/PNEUMONIA/')
  X_train = np.append(norm_images, pneu_images, axis=0)
  y_train = np.append(norm_labels, pneu_labels)
  np.unique(y_train, return_counts=True)
  norm_images_test, norm_labels_test = load_normal('chest_xray/test/NORMAL/')
  pneu_images_test, pneu_labels_test = load_pneumonia('chest_xray/test/PNEUMONIA/')
  X_test = np.append(norm_images_test, pneu_images_test, axis=0)
  y_test = np.append(norm_labels_test, pneu_labels_test)
  y_train = y_train[:, np.newaxis]
  y_test = y_test[:, np.newaxis]
  one_hot_encoder = OneHotEncoder(sparse=False)
  y_train_one_hot = one_hot_encoder.fit_transform(y_train)
  y_test_one_hot = one_hot_encoder.transform(y_test)
  vgg=DenseNet121(input_shape=Image_Size+[3],weights='imagenet',include_top=False)
  for layer in vgg.layers:
    layer.trainable=False
  x=Flatten()(vgg.output)
  x=Dense(256,activation='relu')(x)
  prediction=Dense(2,activation='softmax')(x)
  model=Model(inputs=vgg.input,outputs=prediction)
  model.summary()
  model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])
  train_datagen=ImageDataGenerator(rescale=1./255,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True)
  test_datagen=ImageDataGenerator(rescale=1./255)
  train_datagen.fit(X_train)
  train_gen = train_datagen.flow(X_train, y_train_one_hot, batch_size=32)
  X_test=X_test/255
  from keras.callbacks import EarlyStopping
  from keras.callbacks import ModelCheckpoint
  es = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=2)
  mc = ModelCheckpoint(filepath="specialimp.h5", monitor="val_accuracy", verbose=2,
  save_best_only= True)
  r=model.fit(
  train_gen,
  validation_data=(X_test,y_test_one_hot),
  epochs=35,
  steps_per_epoch=20,
  validation_steps=7,callbacks=[mc])
  plt.figure(figsize=(8,6))
  plt.title('Accuracy scores')
  plt.xlabel('number of epoch')
  plt.ylabel('accuracy')
  plt.plot(r.history['accuracy'])
  plt.plot(r.history['val_accuracy'])
  plt.legend(['accuracy', 'val_accuracy'])
  plt.show()
  plt.figure(figsize=(8,6))
  plt.title('Loss value')
  plt.xlabel('number of epoch')
  plt.ylabel('loss')
  plt.plot(r.history['loss'])
  plt.plot(r.history['val_loss'])
  plt.legend(['loss', 'val_loss'])
  plt.show()
  ## load only the best model
  from keras.models import load_model
  model1 = load_model("specialimp.h5")
  predictions = model1.predict(X_test)
  predictions = one_hot_encoder.inverse_transform(predictions)
  cm = confusion_matrix(y_pred=predictions,y_true=y_test)
  classnames = ['normal', 'pneumonia']
  plt.figure(figsize=(7,7))
  plt.title('Confusion matrix')
  sns.set(font_scale=3)
  sns.heatmap(cm, cbar=False, xticklabels=classnames, yticklabels=classnames, fmt='d',
  annot=True, cmap=plt.cm.Blues)
  plt.xlabel('Predicted')
  plt.ylabel('Actual')
  plt.show()


In [37]:
## Classifer-2
import os
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Model, load_model
from keras.layers import Dense, Input, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
def load_pneumonia(pneu_path):
  pneu_files = np.array(os.listdir(pneu_path))
  pneu_labels = np.array([pneu_file.split('_')[1] for pneu_file in pneu_files])
  pneu_images = []
  for image in tqdm(pneu_files):
    image = cv2.imread(pneu_path + image)
    image = cv2.resize(image, dsize=(224,224))
    pneu_images.append(image)
    pneu_images = np.array(pneu_images)
  return pneu_images, pneu_labels
  bacteria_images, bacteria_labels = load_pneumonia('chest_xray/train/BACTERIA/')
  virus_images, virus_labels = load_pneumonia('chest_xray/train/VIRUS/')
  x_train = np.append(bacteria_images, virus_images, axis=0)
  y_train = np.append(bacteria_labels, virus_labels)
  np.unique(y_train, return_counts=True)
  fig, axes = plt.subplots(ncols=7, nrows=2, figsize=(16, 4))
  indices = np.random.choice(len(x_train), 14)
  counter = 0
  for i in range(2):
    for j in range(7):
      axes[i,j].set_title(y_train[indices[counter]])
      axes[i,j].imshow(x_train[indices[counter]], cmap='gray')
      axes[i,j].get_xaxis().set_visible(False)
      axes[i,j].get_yaxis().set_visible(False)
      counter += 1
  plt.show()
  bacteria_images_test, bacteria_labels_test = load_pneumonia('chest_xray/test/BACTERIA/')
  virus_images_test, virus_labels_test = load_pneumonia('chest_xray/test/VIRUS/')
  x_test = np.append(bacteria_images_test, virus_images_test, axis=0)
  y_test = np.append(bacteria_labels_test, virus_labels_test)
  y_train = y_train[:, np.newaxis]
  y_test = y_test[:, np.newaxis]
  one_hot_encoder = OneHotEncoder(sparse=False)
  y_train_one_hot = one_hot_encoder.fit_transform(y_train)
  y_test_one_hot = one_hot_encoder.transform(y_test)    
  datagen = ImageDataGenerator(
  rotation_range = 10,
  zoom_range = 0.1,
  width_shift_range = 0.1,
  height_shift_range = 0.1)
  datagen.fit(x_train)
  train_gen = datagen.flow(x_train, y_train_one_hot, batch_size=32)
  input1 = Input(shape=(x_train.shape[1], x_train.shape[2], 3))
  cnn = Conv2D(16, (3, 3), activation='relu', strides=(1, 1),
  padding='same')(input1)
  cnn = Conv2D(32, (3, 3), activation='relu', strides=(1, 1),
  padding='same')(cnn)
  cnn = MaxPool2D((2, 2))(cnn)
  cnn = Conv2D(16, (2, 2), activation='relu', strides=(1, 1),
  padding='same')(cnn)
  cnn = Conv2D(32, (2, 2), activation='relu', strides=(1, 1),
  padding='same')(cnn)
  cnn = MaxPool2D((2, 2))(cnn)
  cnn = Flatten()(cnn)
  cnn = Dense(100, activation='relu')(cnn)
  cnn = Dense(50, activation='relu')(cnn)
  output1 = Dense(2, activation='softmax')(cnn)
  model = Model(inputs=input1, outputs=output1)
  model.compile(loss='categorical_crossentropy',
  optimizer='adam', metrics=['accuracy'])
  # implementing early stopping and model check point
  from keras.callbacks import EarlyStopping
  from keras.callbacks import ModelCheckpoint
  es = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=1)
  mc = ModelCheckpoint(filepath="hyu.h5", monitor="val_accuracy", verbose=2,
  save_best_only= True)
  history = model.fit_generator(train_gen,steps_per_epoch=10, epochs=100,
  validation_data=(x_test, y_test_one_hot),callbacks=[mc])
  plt.figure(figsize=(8,6))
  plt.title('Accuracy scores')
  plt.xlabel('number of epoch')
  plt.ylabel('accuracy')
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.legend(['accuracy', 'val_accuracy'])
  plt.show()
  plt.figure(figsize=(8,6))
  plt.title('Loss value')
  plt.xlabel('number of epoch')
  plt.ylabel('loss')
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.legend(['loss', 'val_loss'])
  plt.show()
  ## load only the best model
  from keras.models import load_model
  model1 = load_model("hyu.h5")
  predictions = model.predict(x_test)
  predictions = one_hot_encoder.inverse_transform(predictions)
  cm = confusion_matrix(y_test, predictions)
  classnames = ['bacteria', 'virus']
  plt.figure(figsize=(5,5))
  plt.title('Confusion matrix')
  sns.set(font_scale=3)
  sns.heatmap(cm, cbar=False, xticklabels=classnames, yticklabels=classnames, fmt='d',
  annot=True, cmap=plt.cm.Blues)
  plt.xlabel('Predicted')
  plt.ylabel('Actual')
  plt.show()
  from sklearn.metrics import classification_report as jk
  print(jk(y_test, predictions))

In [40]:
pip install tk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
## User Interface
import tkinter as tk
from tkinter import *
from PIL import Image,ImageTk
from tkinter import filedialog
from keras.models import load_model
from keras.preprocessing import image as pimage
import numpy as np
window= tk.Tk()
window.geometry("700x150")
path=""
window.geometry("1000x675")
window.resizable(False,False)
window.title("Pneumonia Detection And Classification")
window.configure(background='black')

TclError: ignored

# New Section